# Gene comparison

Task description: $\mathbf{a} = [a_1,a_2,a_3,..., a_n]$ and $\mathbf{b} = [b_1,b_2,b_3,..., b_m]$ are two list of genes. $a_i$'s are distinct genes, so are $b_i$'s.
The Pearson correlations between every pair $a_i b_j$ are computed. Note that $a_i$'s and $b_i$'s might not be sorted.
The data is stored at the file `gene_pair` in the format
\begin{align}
&a_1 ~~~ b_1 ~~~ 0.2\\
&a_1 ~~~ b_2 ~~~ 0.3\\
&a_1 ~~~ b_3 ~~~ 0.6\\
&\vdots ~~~~~ \vdots ~~~~~ \vdots
\end{align}
`gene_pair` has $nm$ rows and 3 columnes. Now, we have a feature pair list `feature_pair` in the format,
\begin{align}
& c_1 ~~~ d_1 \\
& c_2 ~~~ d_2 \\
& c_3 ~~~ d_3 \\
&\vdots ~~~~~ \vdots 
\end{align}
`feature_pair` has $l$ rows and 2 columns. 

We want to compare the gene pairs $ab$ with $cd$. If they are the same, we label it with $1$. That is, we want to find all the pairs appearing in both the lists.
To begin comparison, we need to sort the data so that the comparison process has a linear complexcity $O(N)$. The code is
```
sorted(gene_pair, key=itemgetter(0,1))
```



Algorithm: 

1. Sort `gene_pair` by $a_i$ first. If the sorted $a'_i$ have the degenerate elements, $a'_j=a'_{j+1}=...$, sort these elements by $b_i$. For example, $a_i$'s are the ages, and $b_i'$ are the heights. Then, the order is first determined by the ages. If ages are the same, the order among the same ages is determined by height. 
The sorted file `gene_pair_sorted` has the format,
\begin{align}
&a'_1  ~~~ b'_1 ~~~ 0.4\\
&a'_1  ~~~ b'_2 ~~~ 0.2\\
&a'_2  ~~~ b'_3 ~~~ 0.3\\
&a'_2  ~~~ b'_4 ~~~ 0.6\\
&\vdots ~~~~~~~~ \vdots ~~~~~~~ \vdots
\end{align}
where $a'_i$ is sorted, and $b'_i<b'_j$ if $i<j$ when $b'_i$ and
$b'_j$ are of the same $a'_i$.  
2. Do the same as above on `feature_pair` to create `feature_pair_sorted`,
\begin{align}
&c'_1   ~~~ d'_1 \\
&c'_2   ~~~ d'_2 \\
&c'_3   ~~~ d'_3 \\
&\vdots ~~~~~~~~ \vdots 
\end{align}
3. Compare $a'_i $ with $c'_j $. If the same, compare $b'_i $ with $d'_j$. If the same, label $a'_i b'_i$ with 1, else 0.   

In [241]:
import random
import string
import time
import numpy as np
# Function to generate a random string of 'max_length' characters
def generate_random_string(max_length=6):
    length = random.randint(2, max_length)
    return ''.join(random.choices(string.ascii_lowercase, k=length))

# Function to generate a list of 'n' random strings
def generate_random_string_list(n, max_length=6):
    return [generate_random_string(max_length) for _ in range(n)]

In [242]:
n = 1000
m = 1000
l = 100
a = generate_random_string_list(n,2)
b = generate_random_string_list(m,2)
c = generate_random_string_list(l,2)
d = generate_random_string_list(l,2)

gene_pair = []
for i in range(n):
    for j in range(m):
        gene_pair.append([a[i],b[j],np.random.rand()]) # using list
        
feature_pair = []
for i in range(l):
    feature_pair.append([c[i],d[i]])
    
###

    

In [243]:
from operator import itemgetter
gene_pair_sorted = sorted(gene_pair, key=itemgetter(0,1)) ## sort by which column
feature_pair_sorted = sorted(feature_pair, key=itemgetter(0,1)) ## sort by which column

In [244]:
# list1 is gene_pair_sorted, list2 is feature_pair_sorted
#

def compare_sorted_lists(list1, list2): 
    i, j = 0, 0
    list1_TF = []
    list2_TF = []

    while i < len(list1) and j < len(list2):
        if list1[i][0] == list2[j][0]:
            if list1[i][1] == list2[j][1]: ## inner if is for comparing b_i and d_j when a_i = c_j
                list1_TF.append(1) 
                list2_TF.append(1)
                list1[i].append(1)              
                i += 1
                j += 1
            elif list1[i][1] < list2[j][1]:
                list1_TF.append(0)                 
                list1[i].append(0)
                i += 1
            else:   
                list2_TF.append(0)
                j += 1           
        elif list1[i][0] < list2[j][0]:
            list1_TF.append(0)
            list1[i].append(0)
            i += 1
        else:
            list2_TF.append(0)
            j += 1

    # If any list is exhausted, mark the remaining elements as not found
    while i < len(list1):
        list1_TF.append(0)
        list1[i].append(0)
        i += 1

    return list1_TF, list2_TF

def count_ones(lst):
    ones_count = 0
    for item in lst:
        if item == 1:
            ones_count += 1
    return ones_count

In [245]:
l_1, l_2 = compare_sorted_lists(gene_pair_sorted, feature_pair_sorted)
n_ones =count_ones(l_1)
expe_n_ones = (1-(1-1/26/26)**m) * (1-(1-1/26/26)**n) * l

In [246]:
print(n_ones,end='\n')
print(expe_n_ones,end='\n')

68
59.66789623942363


In [247]:
indices_1 = [i for i, x in enumerate(l_1) if x == 1]
indices_2 = [i for i, x in enumerate(l_2) if x == 1]

In [248]:
for i in indices_1:
    print(i, gene_pair_sorted[i],end='\n')

3714 ['ae', 'iv', 0.6804277951544807, 1]
7330 ['ag', 'ih', 0.17221952864736612, 1]
31678 ['ay', 'il', 0.5998057541357955, 1]
41089 ['be', 'cf', 0.24114316123793011, 1]
78330 ['cc', 'ei', 0.66257437302065, 1]
84677 ['cf', 'op', 0.7325483188992382, 1]
115030 ['db', 'ak', 0.38341297805417385, 1]
124084 ['de', 'ax', 0.5895757954289582, 1]
145489 ['dp', 'mw', 0.7444723345725158, 1]
162668 ['ee', 'rj', 0.3206728761162033, 1]
171227 ['el', 'fu', 0.1478502065796219, 1]
171816 ['el', 'vg', 0.3185731480082675, 1]
172049 ['em', 'bc', 0.8909233862247616, 1]
234837 ['gb', 'ha', 0.15229722758234254, 1]
235665 ['gb', 'om', 0.8141821215078235, 1]
251799 ['gp', 'uw', 0.6244758285926428, 1]
257231 ['gu', 'fy', 0.27688786749626404, 1]
270660 ['hc', 're', 0.8443245119974958, 1]
278304 ['hg', 'hn', 0.43614483757894174, 1]
279983 ['hh', 'zn', 0.8859988695144402, 1]
283312 ['hj', 'pb', 0.14773672503717405, 1]
287484 ['hl', 'mt', 0.7679406787345608, 1]
298155 ['hs', 'dy', 0.09969169844487702, 1]
308147 ['id',

In [249]:
for i in indices_2:
    print(i, feature_pair_sorted[i],end='\n')

0 ['ae', 'iv']
1 ['ag', 'ih']
4 ['ay', 'il']
5 ['be', 'cf']
7 ['cc', 'ei']
9 ['cf', 'op']
10 ['db', 'ak']
11 ['de', 'ax']
12 ['dp', 'mw']
14 ['ee', 'rj']
16 ['el', 'fu']
17 ['el', 'vg']
18 ['em', 'bc']
22 ['gb', 'ha']
23 ['gb', 'om']
25 ['gp', 'uw']
26 ['gu', 'fy']
27 ['hc', 're']
28 ['hg', 'hn']
29 ['hh', 'zn']
30 ['hj', 'pb']
31 ['hl', 'mt']
32 ['hs', 'dy']
33 ['id', 'bc']
34 ['ih', 'jx']
36 ['jj', 'sz']
39 ['jr', 'zq']
40 ['kc', 'ej']
41 ['kg', 'to']
45 ['mb', 'mm']
46 ['mm', 'pu']
47 ['nb', 'ux']
48 ['nj', 'vy']
49 ['ny', 'nt']
50 ['oa', 'vv']
51 ['oc', 'js']
52 ['of', 'op']
53 ['on', 'wk']
55 ['or', 'ka']
57 ['pq', 'mu']
58 ['qi', 'vb']
59 ['qo', 'iw']
60 ['qs', 'nu']
61 ['qw', 'bm']
62 ['qy', 'zz']
63 ['rg', 'ub']
70 ['ts', 'gs']
71 ['tv', 'ci']
73 ['ul', 'ww']
74 ['uw', 'ix']
75 ['vf', 'bf']
76 ['vi', 'sk']
78 ['vv', 'bc']
79 ['wf', 'fp']
80 ['wh', 'nf']
82 ['wl', 'gm']
83 ['wn', 'lo']
84 ['wt', 'ql']
86 ['ww', 'dj']
87 ['ww', 'qu']
91 ['xw', 'mz']
92 ['yg', 'gq']
93 ['yj', 'ge'

In [250]:
print(len(indices_1))
print(len(indices_2))

68
68
